In [3]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import pickle

# Load the ratings dataset
ratings_df = pd.read_csv(r"C:\Users\ASUS\Desktop\documents canada\ratings_merged.csv")

# Load the trained model
model = pickle.load(open('model.pkl', 'rb'))

# Function to get top N recommendations for a user
def get_top_n_recommendations(user_id, model, ratings_df, n=10):
    # Get a list of all movie IDs
    all_movie_ids = ratings_df['movieId'].unique()
    
    # Exclude movies already rated by the user
    rated_movie_ids = ratings_df.loc[ratings_df['userId'] == user_id, 'movieId'].unique()
    unrated_movie_ids = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movie_ids]
    
    # Predict ratings for unrated movies
    unrated_predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movie_ids]
    
    # Sort the predictions by estimated rating in descending order
    top_n_predictions = sorted(unrated_predictions, key=lambda x: x.est, reverse=True)[:n]
    
    # Get movie titles corresponding to the top-N predicted movie IDs
    top_n_movie_titles = [ratings_df.loc[ratings_df['movieId'] == pred.iid, 'original_title'].iloc[0] for pred in top_n_predictions]
    
    return top_n_movie_titles

# Function to handle button click event
def get_recommendations():
    try:
        user_id = int(user_id_entry.get())  # Get user ID entered by the user
        recommendations = get_top_n_recommendations(user_id, model, ratings_df)  # Get movie recommendations for the user
        result_text = "\n".join(recommendations)
        result_label.config(text=result_text)  # Display recommendations
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Create Tkinter window
root = tk.Tk()
root.title("Movie Recommendation System")

# User ID entry
user_id_label = tk.Label(root, text="Enter User ID:")
user_id_label.pack()
user_id_entry = tk.Entry(root)
user_id_entry.pack()

# Button to get recommendations
get_recommendations_button = tk.Button(root, text="Get Recommendations", command=get_recommendations)
get_recommendations_button.pack()

# Label to display recommendations
result_label = tk.Label(root, text="")
result_label.pack()

# Run Tkinter event loop
root.mainloop()
